# Implementation of SegNet on Drone Images 

This notebook recreates the procedure of data preperation and data loading and also initializing, training and evaluating SegNet on this data.

As the code is organized in modules, large portions of the produced code is not contained directly herein, but can be consulted by opening the `.py` files in the `src/`-folder.

NOTE: Because the full network takes upwards of 30 GB of memory to train and evaluate, a simplified version is used here. The full version can be accessed by setting the `use_simple` variable to `False`. Runtime evaluation is also disabled by default to save memory. It is explained how to enable it further down.

In [1]:
!echo "Make sure that you are in /src before running this notebook"
!pwd  # Prints current path (Linux and MacOS only)

# For running on google colab
# %cd /content
# !git clone https://github.com/sorenmulli/alpha_cellari_imageseg.git
# %cd alpha_cellari_imageseg/src
# %mkdir logs
# %pwd

"Make sure that you are in /src before running this notebook"
'pwd' is not recognized as an internal or external command,
operable program or batch file.
2020-01-03 20:55:43.484879	Running full training loop

Traceback (most recent call last):
  File "train.py", line 156, in <module>
    trainer = Trainer("local_data/prep_out.json", logger)
  File "train.py", line 32, in __init__
    with open(json_path, encoding="utf-8") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'local_data/prep_out.json'


## Data preperation

Downloads and prepares data. This will take a number of minutes to run and requires a little over a gigabyte of storage.

In [0]:
!python -m pip install wget --user
!python data_prepper.py

## Preparing network training


In [1]:
from logger import Logger
from train import Trainer
from model_test import Tester
from augment import Augmenter, AugmentationConfig, flip_lr, flip_tb
import torch

print("Configuring parameters...")

# If True, a simpler version of segnet is used
use_simple = True

# Configuration of network architechture
architecture = {
		"kernel_size":  3,
		"padding": 1, 
		"stride": 1,
		"pool_dims": (2, 2),
		"probs": 0.1,
		"reduce_complexity": use_simple,
}

learning_rate = 1.5e-4
batch_size = 3
epochs = 3000

# Configuration of data augmentation
# Reducing cropsize significantly reduces memory usage and training time
augmentations = AugmentationConfig(
    augments =  [flip_lr, flip_tb],  
    cropsize = (350, 350),
    augment_p = [0.5, 0.5]
)

torch.cuda.empty_cache()
logger = Logger("logs/train_run.log", "Running full training loop")

print("Done configuring")

Configuring parameters...
2020-01-03 20:52:00.773290	Running full training loop

Done configuring


## Training the Network

The following code trains the network.

Because we had access to significant compute recourses, we did not optimize the loss calculations for memory. It is therefore not possible to run evaluation on most machines, so it is disabled by default. It can be enabled by setting the `with_loss` argument to `True`. The loss curve is available in the report.

In [2]:
trainer = Trainer("local_data/prep_out.json", logger)
net = trainer.model_trainer(architecture, learning_rate, augmentations, epochs, batch_size, val_every = 25, with_loss = False, save_every = 500)
net.save('model')

FileNotFoundError: [Errno 2] No such file or directory: 'local_data/prep_out.json'


## Evaluating the Network

In [0]:
from model import Net
# Load model or use existing
# net = Net.from_model("model")
tester = Tester("local_data/prep_out.json", logger)
tester.test_model(net, "local_data/test")


## Reconstructing the image

In the following, a complete forwarding is carried out using oversampling to prevent borders at the croppings.

In [0]:
from forward_passer import full_forward
full_forward(net, None, True, "local_data/full-forward.png")